<a href="https://colab.research.google.com/github/ENVIRON-ENERGY716/Fall2023/blob/main/Labs/Lab10/Lab10_NetworkFlow_TransportationProblem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 11 - More Network Models with Pyomo**


Learning goals for Lab 11:
* Implement other types of Network Models - Max flow;
* Implement other types of Network Models - Shortest path.


## Initializing

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/')

Mounted at /content/drive


Installing Pyomo and solver. Recall for teh shortest path example we have binary variables so we will need to use another solver. Instead of installing glpk, thsi time we will install COIN-OR CBC. \\
COIN-OR CBC is a multi-threaded open-source Coin-or branch and cut **mixed-integer linear programming solver**. CBC is generally a good choice for a general purpose MILP solver for medium to large scale problems.

In [2]:
!pip install pyomo
#!apt-get install -y -qq glpk-utils
!apt-get install -y -qq coinor-cbc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.8 MB/s eta 0:00:00
Selecting previously unselected package coinor-libcoinutils3v5:amd64.
(Reading database ... 120874 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-2_amd64.deb ...
Unpacking coinor-libcoinutils3v5:amd64 (2.11.4+repack1-2) ...
Selecting previously unselected package coinor-libosi1v5:amd64.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-2_amd64.deb ...
Unpacking coinor-libosi1v5:amd64 (0.108.6+repack1-2) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1:amd64.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-3_amd64.deb ...
Unpacking coinor-libcgl1:amd64 (0.60.3+repack1-3

Importing pyomo and cbc solver.

In [3]:
from pyomo.environ import *
#Import solver
opt=SolverFactory('cbc',executable='/usr/bin/cbc')

## Exercise 1: Maximum Flow Model - Natural Gas company

Let's start by writing the max flow problem.

**Sets** \\
$N$: set of nodes \\
$K$: set of nodes without source and sink nodes \\
$A$: set of arcs $(ij)$ \\

**Parameters** \\
$u_{ij}$: flow capacity for arc $(ij)$

**Decision Variable** \\
$x_{ij}$: how much flow on arc $(ij)$ - any value from 0 to $u_{ij}$

**Model** \\
$ max \ \sum_{j \in N} x_{1j}$ \\
$ s.t.$
$ \ \sum_{j \in N} x_{ij} = \sum_{j \in N} x_{ji} \quad \forall i \in K $ \\
$ \quad \quad \sum_{j \in N} x_{1j} = \sum_{j \in N} x_{j10} $ \\
$ \quad \quad x_{ij} \leq u_{ij} \quad \forall (ij) \in A $ \\
$ \quad \quad x_{ij} \geq 0 \quad \forall (ij) \in A $


### Implementing a representative model.

In [31]:
model=ConcreteModel()

model.Nodes=Set(initialize=range(1,8))

model.first=1
model.last=7
#defining set of nodes without origin and destination
model.NodesK=Set(within=model.Nodes,initialize=range(2,7))



model.Arcs=Set(within=model.Nodes*model.Nodes,
                initialize=[(1,2),(1,3),
                            (2,4),(2,5),
                            (3,4),(3,6),
                            (4,5),(4,6),(4,7),
                            (5,7),
                            (6,7)])

model.Arcs.pprint()

#Add parameter
model.FlowCap=Param(model.Arcs,
                  initialize={(1,2):9,(1,3):6,
                              (2,4):5,(2,5):4,
                              (3,4):2,(3,6):4,
                              (4,5):4,(4,6):5,(4,7):7,
                              (5,7):3,
                              (6,7):6})
#Add dec variables
model.x=Var(model.Arcs,domain=NonNegativeReals)

Arcs : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain      : Size : Members
    None :     2 : Arcs_domain :   11 : {(1, 2), (1, 3), (2, 4), (2, 5), (3, 4), (3, 6), (4, 5), (4, 6), (4, 7), (5, 7), (6, 7)}


In [32]:
#Adding objective function
def max_flow(model):
    #
model.maxflow=Objective(rule=max_flow, sense=maximize)

print(model.maxflow.expr)


x[1,2] + x[1,3]


In [33]:
#Adding constraints
#Flow balance transhipment nodes - for all nodes in K
def flow_bal(model, n):
    #
model.flowbalance = Constraint(model.NodesK, rule=flow_bal)

#printing constraints
for n in model.NodesK:
  print(model.flowbalance[n].expr)

x[1,2]  ==  x[2,4] + x[2,5]
x[1,3]  ==  x[3,4] + x[3,6]
x[2,4] + x[3,4]  ==  x[4,5] + x[4,6] + x[4,7]
x[2,5] + x[4,5]  ==  x[5,7]
x[3,6] + x[4,6]  ==  x[6,7]


In [34]:
#Adding constraints
#flow origin = flow destination - only one
def orig_dest(model):
    #
model.origdest = Constraint(rule=orig_dest)

#printing org = dest constraints
print(model.origdest.expr)

x[1,2] + x[1,3]  ==  x[4,7] + x[5,7] + x[6,7]


In [35]:
#Adding constraints
#arc flow capacity
def flow_cap(model,i,j):
    #
model.flowcap = Constraint(model.Arcs,rule=flow_cap)

#printing max flow constraints
for (i,j) in model.Arcs:
  print(model.flowcap[i,j].expr)

x[1,2]  <=  9
x[1,3]  <=  6
x[2,4]  <=  5
x[2,5]  <=  4
x[3,4]  <=  2
x[3,6]  <=  4
x[4,5]  <=  4
x[4,6]  <=  5
x[4,7]  <=  7
x[5,7]  <=  3
x[6,7]  <=  6


In [36]:
#Solving model
opt.solve(model)

#Print results
print("Max Flow from 1 to 10 =",model.maxflow())
print("Decision Variables")
for a in model.Arcs:
    print(model.x[a],model.x[a].value)

Max Flow from 1 to 10 = 14.0
Decision Variables
x[1,2] 8.0
x[1,3] 6.0
x[2,4] 5.0
x[2,5] 3.0
x[3,4] 2.0
x[3,6] 4.0
x[4,5] 0.0
x[4,6] 0.0
x[4,7] 7.0
x[5,7] 3.0
x[6,7] 4.0


## Exercise 2: Shortest Path - min cost

Let's start by writing the shortest path model formulation.

**Sets** \\
$N$: set of nodes \\
$K$: set of nodes without source and sink \\
$A$: set of arcs $(ij)$ \\

**Parameters** \\
$c_{ij}$: cost for using arc $ij$

**Decision Variable** \\
$x_{ij}$: 1 if arc $(ij)$ is being used, 0 o.w. - binary

**Model** \\
$ min \ \sum_{(ij) \in A} c_{ij}*x_{ij}$ \\
$ s.t.$
$ \ \sum_{j \in N} x_{ij} = \sum_{j \in N} x_{ji} \quad \forall i \in K $ \\
$ \quad \quad \sum_{j \in N} x_{1j} = 1 $ \\
$ \quad \quad \sum_{i \in N} x_{i10} = 1 $ \\
$ \quad \quad x_{ij} \in \{0,1\} \quad \forall (ij) \in A $


### Part a: Implementing model from lectures.

In [20]:
model2=ConcreteModel()

model2.Nodes=Set(initialize=range(1,8))
model2.first=1
model2.last=7

model2.Arcs=Set(within=model2.Nodes*model2.Nodes,
                initialize=[(1,2),(1,3),
                            (2,4),(2,5),
                            (3,4),(3,6),
                            (4,5),(4,6),(4,7),
                            (5,7),
                            (6,7)])

model2.Arcs.pprint()

#Add parameter
model2.cost=Param(model2.Arcs,
                  initialize={(1,2):9,(1,3):6,
                              (2,4):5,(2,5):4,
                              (3,4):2,(3,6):4,
                              (4,5):4,(4,6):5,(4,7):7,
                              (5,7):3,
                              (6,7):6})

#Add dec variables
#arc being used or not - decision variable xij
model2.x=Var(model2.Arcs,domain=Boolean)

Arcs : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain      : Size : Members
    None :     2 : Arcs_domain :   11 : {(1, 2), (1, 3), (2, 4), (2, 5), (3, 4), (3, 6), (4, 5), (4, 6), (4, 7), (5, 7), (6, 7)}


In [21]:
#Adding objective function
def min_path(model2):
    #
model2.shortest=Objective(rule=min_path, sense=minimize)

print("\nObjective Function")
print(model2.shortest.expr)


Objective Function
9*x[1,2] + 6*x[1,3] + 5*x[2,4] + 4*x[2,5] + 2*x[3,4] + 4*x[3,6] + 4*x[4,5] + 5*x[4,6] + 7*x[4,7] + 3*x[5,7] + 6*x[6,7]


In [22]:
#Adding constraints
def flow_rule(model2, n):
    #
model2.flow = Constraint(model2.Nodes, rule=flow_rule)

print("Constraints")
for n in model2.Nodes:
  print(model2.flow[n].expr)

Constraints
x[1,2] + x[1,3]  ==  1
x[1,2]  ==  x[2,4] + x[2,5]
x[1,3]  ==  x[3,4] + x[3,6]
x[2,4] + x[3,4]  ==  x[4,5] + x[4,6] + x[4,7]
x[2,5] + x[4,5]  ==  x[5,7]
x[3,6] + x[4,6]  ==  x[6,7]
x[4,7] + x[5,7] + x[6,7]  ==  1


### Part b: Solving the model

In [24]:
opt.solve(model2)

#Print results
print("Lowest cost from 1 to 7 =",model2.shortest())
print("Decision Variables")
for a in model2.Arcs:
    print(model2.x[a],model2.x[a].value)

Lowest cost from 1 to 7 = 15.0
Decision Variables
x[1,2] 0.0
x[1,3] 1.0
x[2,4] 0.0
x[2,5] 0.0
x[3,4] 1.0
x[3,6] 0.0
x[4,5] 1.0
x[4,6] 0.0
x[4,7] 0.0
x[5,7] 1.0
x[6,7] 0.0
